## BeerAdvocate: Descriptive Analysis and Processing

---

**Processing and Transformations Summary of "users.csv"**

- **Renaming Columns**: "nbr_ratings" to "n_ratings", "nbr_reviews" to "n_reviews", "user_id" to "uid", "user_name" to "username".
- **Converting Data Types**: "n_ratings", "n_reviews" to int & "username", "uid", "location" to str.
- **Converting "nan" to None**: in "location".
- **Creating an additional "country" column from "location"**: by extracting the country name from the location (example: "United States, Texas" location belongs to country "United States").
- **Converting the "joined" column values**: float values (time in seconds, reference is Jan.1st 1970 at 00:00) to pandas.Timestamp. Implicit conversion of NaN to NaT. 

---

**Descriptive Analysis Summary of "users.csv"**

* **Missing values**: none, except for "location" that may contain "nan" (previously converted to None) and "joined" that may contain NaN (previously implictely converted to NaT).

<br>

* **users**:
  - 153'704 unique users.
  - $\approx$ 20.3% of users are of unknown location.
  - most users are located in English speaking countries. top 5 countries in terms of number of users:
  
  | country | wrt to all users | wrt users with known location |
  | :- | :-: | :-: |
  | United States | 73.6% | 92.4% |
  | Canada | 2.1% | 2.7% |
  | England | 0.5% | 0.6% | 
  | Australia | 0.3% | 0.4% |
  | Germany | 0.2% | 0.3% |

<br>

* **ratings**:
  - 8'393'032 ratings in total.
  - heavy-tailed distribution of the number of ratings per user.
  - 50% of users have at most 3 ratings.
  - the mean number of ratings $\approx 54.6$ is quite low while the standard deviation $\approx 252.38$ is very high.
  - most rtings are given by users located in English speaking countries. Top 5 countries in terms of the number of ratings: 

  | country | wrt to all users | wrt users with known location |
  | :- | :-: | :-: |
  | United States | 87% | 92.5% |
  | Canada | 3.2% | 3.4% |
  | England | 0.4% | 0.5% | 
  | Australia | 0.4% | 0.4% |
  | Sweden | 0.3% | 0.3% |

<br>

* **reviews**:
  - reviews in total.
  - distribtion ?
  - 50% of users have 0 reviews.
  - the mean number of reviews $\approx 16.84$ is quite low while the standard deviation $\approx 139.84$ is very high.
  - most reviews are given by users from English speaking countries (US, Canada, Australia, England). Top 5 countries in terms of the number of reviews:
  
  | country | wrt to all users | wrt users with known location |
  | :- | :-: | :-: |
  | United States | 86.5% | 89.8% |
  | Canada | 4.8% | 5% |
  | Australia | 1% | 1% | 
  | England | 0.8% | 0.8% |
  | Netherlands | 0.3% | 0.3% |

<br>

* **joined**:
  - earliest year of joining: 1996.
  - latest year of joining: 2017 (date of the end of the collection of data, which happened between 2001 and 2017).
  - there are only 2652 users with unknown joining date.
  - the yearly by month distribution of users joining the plateform is not disrupted throughout most of the years (starting 2001 until 2017). 

<br>

* **country**:
  - users are located in 145 different countries.

---

In [ ]:
from setup import dask_init, dask_shutdown
import dask.distributed as dd
import dask.dataframe as ddf

import pandas as pd
import numpy as np

import ingestion as ing
import processing as prc

import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

**setup**

In [ ]:
# PANDAS SETUP
pd.options.mode.use_inf_as_na = True

# DASK SETUP
client, cluster = dask_init()
cluster

### BeerAdvocate - users.csv

**[I] Loading the Data**

In [ ]:
users_cols = [
    "nbr_ratings",
    "nbr_reviews",
    "user_id",
    "user_name",
    "joined",
    "location"]

users_df = ing.read_csv(
    path=ing.build_path(folderind="ba", fileind="users"),
    assume_missing=True,
    keepcols=users_cols)

users_df.head(5)

**[II] Data Transformation**

Renaming the columns

In [ ]:
cols_renaming = {
    "nbr_ratings": "n_ratings",
    "nbr_reviews": "n_reviews",
    "user_id": "uid",
    "user_name": "username"
}

users_df = users_df.rename(columns=cols_renaming)
users_df.columns

Converting data types

In [ ]:
users_df.dtypes

In [ ]:
users_dtypes = {
    "n_ratings": np.int32, 
    "n_reviews": np.int32, 
    "uid": str, 
    "username": str,
    "location": str
}

users_df = users_df.astype(users_dtypes)
users_df.dtypes

Converting "nan" strings in "location" to None

In [ ]:
users_df["location"] = users_df.location.apply(prc.str_nan_to_none)

Converting the "joined" column values (time in seconds) to Timestamps

In [ ]:
users_df["joined"] = ddf.to_datetime(users_df.joined, unit="s")
users_df.head(5)

In [ ]:
# CHECK
print("CHECK: max date year must be at most 2017, the last year data was collected.")
users_df.joined.max()

Appending the Country of Each Territory

In [ ]:
users_df["country"] = users_df["location"].apply(lambda territory: prc.get_country(territory))
users_df.head(5)

**[III] Descriptive Statistics & Distributions**

In [ ]:
users_df.describe()

user_id

In [ ]:
print("Is any user ID missing ?")
users_df.uid.isna().any()

In [ ]:
print("How many unique users ?")
n_users = len(users_df.uid.unique())
n_users

location

In [ ]:
print("Is there a user with unknown location ?")
users_df.location.isna().any()

In [ ]:
print("How many users have an unknown location ?")
n_users_unknown_loc = users_df.location.isna().sum()
n_users_unknown_loc

In [ ]:
print("What fraction do these users make up of the total number of users ?")
n_users_unknown_loc / n_users

In [ ]:
print("Where are all users located ?")
all_countries = users_df.country[~users_df.country.isna()].unique()
all_countries.sort()
all_countries

In [ ]:
print("How many different locations are there ?")
len(all_countries)

In [ ]:
n_users_per_country_wout_us = users_df[\
    (~users_df.country.isna()) &\
    (users_df.country != "United States")].groupby("country", as_index=False).size()

fig, ax = plt.subplots(1,1)
ax.set_title("Number of Users Across All Countries except the US")
ax.set_ylabel("users' count")
ax.set_xlabel("country (represented via its index entry)")
sns.scatterplot(data=n_users_per_country_wout_us, ax=ax, s=10)

In [ ]:
print("List of countries with at least 100 users")
n_users_per_country = users_df.groupby("country", as_index=False).size()
countries_atleast_100users = n_users_per_country[n_users_per_country["size"] >= 100]
countries_atleast_100users["users_frac"] = countries_atleast_100users["size"] / users_df.location[~ users_df.location.isna()].size
countries_atleast_100users = countries_atleast_100users.sort_values(by="size", ascending=False)
countries_atleast_100users

n_ratings

In [ ]:
print("Is there a user with unknown number of ratings ?")
users_df.n_ratings.isna().any()

In [ ]:
print("Total number of ratings")
total_n_ratings = users_df.n_ratings.sum()
total_n_ratings

In [ ]:
fig, ax = plt.subplots(1,1)
ax.set_title("Distribution of Ratings Across All Users")
ax.set_ylabel("occurences")
ax.set_xlabel("ratings")
sns.histplot(data=users_df, x="n_ratings", ax=ax, bins=20, log_scale=True)


In [ ]:
n_ratings_per_country = users_df[\
    (~users_df.country.isna()) &\
    (users_df.country != "United States")].groupby("country", as_index=False).agg({"n_ratings": np.sum})

fig, ax = plt.subplots(1,1)
ax.set_title("Number of Ratings Across All Countries except the US")
ax.set_ylabel("ratings' count")
ax.set_xlabel("country (represented by its entry index)")
sns.scatterplot(data=n_ratings_per_country, ax=ax, s=10)

In [ ]:
print("List of countries with at least 10'000 ratings")
min_n_ratings = 10_000
n_ratings_per_country = users_df[~users_df.country.isna()].groupby("country", as_index=False).agg({"n_ratings": np.sum})
countries_atleast_10Kratings = n_ratings_per_country[n_ratings_per_country.n_ratings >= min_n_ratings]
countries_atleast_10Kratings["ratings_frac"] = countries_atleast_10Kratings.n_ratings / users_df[~users_df.country.isna()].n_ratings.sum()
countries_atleast_10Kratings = countries_atleast_10Kratings.sort_values(by="n_ratings", ascending=False)
countries_atleast_10Kratings

n_reviews

In [ ]:
print("Is there a user with unknown number of reviews ?")
users_df.n_reviews.isna().any()

In [ ]:
print("Total number of reviews")
total_n_reviews = users_df.n_reviews.sum()
total_n_reviews

In [ ]:
fig, ax = plt.subplots(1,1)
ax.set_title("Distribution of Reviews Across All Users")
ax.set_ylabel("occurences")
ax.set_xlabel("reviews")
sns.histplot(data=users_df, x="n_reviews", ax=ax, bins=20, log_scale=True)

In [ ]:
print("List of countries with at least 5'000 reviews")
min_n_reviews = 5_000
n_reviews_per_country = users_df[~users_df.country.isna()].groupby("country", as_index=False).agg({"n_reviews": np.sum})
countries_atleast_5Kreviews = n_reviews_per_country[n_reviews_per_country.n_reviews >= min_n_reviews]
countries_atleast_5Kreviews["reviews_frac"] = countries_atleast_5Kreviews.n_reviews /\
    users_df.n_reviews[~users_df.country.isna()].sum()
countries_atleast_5Kreviews = countries_atleast_5Kreviews.sort_values(by="n_reviews", ascending=False)
countries_atleast_5Kreviews

joined

In [ ]:
print("Is there any missing joining date ?")
users_df.joined.isna().any()

In [ ]:
print("How many users have unknown joining date ?")
users_df.joined.isna().sum()

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(12,10))
plt.subplots_adjust(wspace=1, hspace=1)
fig.suptitle("Distribution of Users Joining the Plateform from 1996 to 2017 Broken Down by Month")
users_joined_by_year = users_df.groupby(users_df.joined.dt.year, as_index=False)
for i, (year, by_year) in enumerate(users_joined_by_year):
    a = int(i/4); b = i % 4
    ax = axs[a][b]
    ax.set_title("year %d"%(year))
    users_joined_by_month = by_year.groupby(by_year.joined.dt.month, as_index=False).size()
    sns.barplot(data=users_joined_by_month, x="joined", y="size", color="blue", ax=ax, orient="v")